In [141]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from scipy import sparse

In [142]:
book_df = pd.read_csv('data/Books.csv')
ratings_df = pd.read_csv('data/Ratings.csv')
user_df = pd.read_csv('data/Users.csv')
ratings_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [143]:
ratings_df = ratings_df[ratings_df['Book-Rating'] > 0]

book_counts = ratings_df['ISBN'].value_counts()
popular_books = list(book_counts[book_counts >= 10].index)

ratings_count = ratings_df['User-ID'].value_counts()
users_with_many_ratings = list(ratings_count[ratings_count >= 10].index)

ratings_df = ratings_df[(ratings_df['ISBN'].isin(popular_books)) & ratings_df['User-ID'].isin(users_with_many_ratings)]
ratings_df.sort_values('User-ID', ascending=True)

,User-ID,ISBN,Book-Rating,Location,Age
9914,242,0553278223,10,"neuffen, baden-wuerttemberg, germany",37.0
9931,242,3257233051,9,"neuffen, baden-wuerttemberg, germany",37.0
10009,243,0446606383,6,"arden hills, minnesota, usa",NaN
10035,243,155874262X,5,"arden hills, minnesota, usa",NaN
10030,243,0786863986,5,"arden hills, minnesota, usa",NaN
...,...,...,...,...,...
9457,278843,0060173289,9,"pismo beach, california, usa",28.0
9480,278843,0440211263,7,"pismo beach, california, usa",28.0
9536,278851,0440486599,5,"dallas, texas, usa",33.0
9545,278851,0894803700,5,"dallas, texas, usa",33.0


In [144]:
book_user_rating = book_df.merge(ratings_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)
book_user_rating.head()

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0399135782,The Kitchen God's Wife,Amy Tan,11676,9
1,0399135782,The Kitchen God's Wife,Amy Tan,29526,9
2,0399135782,The Kitchen God's Wife,Amy Tan,46398,9
3,0399135782,The Kitchen God's Wife,Amy Tan,67958,10
4,0399135782,The Kitchen God's Wife,Amy Tan,115097,7


In [145]:
d = {}
for i, j in enumerate(book_user_rating.ISBN.unique()):
    d[j] = i
book_user_rating["unique_id_book"] = book_user_rating["ISBN"].map(d)
book_user_rating.reset_index(drop=True, inplace=True)
book_user_rating.head()


,ISBN,Book-Title,Book-Author,User-ID,Book-Rating,unique_id_book
0,0399135782,The Kitchen God's Wife,Amy Tan,11676,9,0
1,0399135782,The Kitchen God's Wife,Amy Tan,29526,9,0
2,0399135782,The Kitchen God's Wife,Amy Tan,46398,9,0
3,0399135782,The Kitchen God's Wife,Amy Tan,67958,10,0
4,0399135782,The Kitchen God's Wife,Amy Tan,115097,7,0


test
s1
Строим разреженную матрицу

In [146]:
df_pivot = book_user_rating.pivot(index='unique_id_book',
                                  columns='User-ID',
                                  values='Book-Rating').fillna(0)

In [151]:
df_pivot.head()

User-ID,unique_id_book,242,243,254,388,446,503,505,507,625,...,278314,278356,278390,278418,278535,278554,278582,278633,278843,278851
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [152]:
df_pivot.shape


(5441, 6781)

In [153]:
df_sparse = sparse.csr_matrix(df_pivot)
df_pivot.reset_index(inplace=True)

In [154]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(df_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [155]:
ISBN = '0399135782'
book_unique_id = book_user_rating[book_user_rating['ISBN']==ISBN]['unique_id_book'].unique()[0]
book_unique_id

0

In [157]:
def get_book_recommendation(ISBN, n_recommend: int):
    book_unique_id = book_user_rating[book_user_rating['ISBN']==ISBN]['unique_id_book'].unique()[0]
    distances, indices = knn.kneighbors(df_sparse[book_unique_id], n_neighbors=n_recommend)
    rec_book_indices = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())),
                                key=lambda x: x[1])[:0:-1]
    recommendations = []
    for val in rec_book_indices:
        book_idx = book_user_rating.iloc[val[0]]['ISBN']
        idx = book_df[book_df['ISBN'] == book_idx].index
        recommendations.append({'ISBN':book_df.iloc[idx]['ISBN'].values[0],
                                'Book-Title':book_df.iloc[idx]['Book-Title'].values[0],
                                'Book-Author':book_df.iloc[idx]['Book-Author'].values[0],
                                'Year-Of-Publication':book_df.iloc[idx]['Year-Of-Publication'].values[0],
                                'Publisher':book_df.iloc[idx]['Publisher'].values[0],
                                'Distance':val[1]})
    df = pd.DataFrame(recommendations, index=range(1, n_recommend))
    return df

def recommend(user_id, n_recommend=5):
    user_df = book_user_rating[book_user_rating['User-ID'] == user_id]
    max_rating = user_df['Book-Rating'].max()
    max_rated_book = user_df[user_df['Book-Rating'] == max_rating]
    recommendations = get_book_recommendation(max_rated_book['ISBN'], n_recommend)

    return recommendations

In [158]:
recommend(8, n_recommend = 10 + 1)


ValueError: Can only compare identically-labeled Series objects

In [107]:
df_sparse[1, :]

<1x8709 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>